In [5]:
import pandas as pd
import zipfile
import os
import re
import numpy as np

In [9]:
#Descomprimimos
destino = 'datasets/'

if not os.path.exists(destino):
    os.makedirs(destino)

with zipfile.ZipFile("univaciada_1.zip", 'r') as archivo_zip:
    archivo_zip.extractall(destino)

with zipfile.ZipFile("univaciada_2.zip", 'r') as archivo_zip:
    archivo_zip.extractall(destino)

with zipfile.ZipFile("univaciada_3.zip", 'r') as archivo_zip:
    archivo_zip.extractall(destino)

with zipfile.ZipFile("INe.zip", 'r') as archivo_zip:
    archivo_zip.extractall()

In [44]:
#Preprocesamiento
ficheros = os.listdir(destino)
ficheros

#Solo cargamos las columnas necesarias
columnas_interes = ["curso_academico", 'des_municipio_residencia',"cod_tipo_estudio", "cod_municipio_residencia",  "lat_municipio_residencia", "lon_municipio_residencia"]

dataframes = []

#Limpieza de datos, nos quedamos solo con estudiantes de Grado españoles. Añadimos una columna con la universidad del estudiante.
for fichero in ficheros:
    try:
        df = pd.read_csv(destino+"/"+fichero, usecols=columnas_interes, low_memory=False)
        df = df[df["cod_tipo_estudio"] == 'G']  #Estudiantes de Grado
        df = df[df['des_municipio_residencia'] != 'MUNICIPIO EXTRANJERO PERTENECIENTE A LA U.E.']
        df = df[df['des_municipio_residencia'] != 'MUNICIPIO EXTRANJERO NO PERTENECIENTE A LA U.E.']
        df = df.drop(columns=["des_municipio_residencia"])
        df = df.dropna()
        df['cod_municipio_residencia'] = df['cod_municipio_residencia'].astype('int')
        nombre = fichero.split('/')[-1]
        universidad = re.match(r'^([^-]+)', nombre).group(1)
        df['universidad'] = universidad
        dataframes.append(df)  
    except Exception as e:
        print(f"Error al cargar el archivo desde {fichero}: {e}")

#Concatenamos
df_matriculas = pd.concat(dataframes, ignore_index=True)

#Calculamos el tamaño de la burbuja normalizando los datos
df_matriculas['count'] = 1
df_grouped = df_matriculas.groupby(['lat_municipio_residencia', 'lon_municipio_residencia', "cod_municipio_residencia",'universidad', 'curso_academico'], as_index=False).agg({
    'count': 'sum'
})

min_count = df_grouped['count'].min()
max_count = df_grouped['count'].max()
df_grouped['scaled_count'] = 1 + 19 * (df_grouped['count'] - min_count) / (max_count - min_count)
df_grouped['log_count'] = df_grouped['count'].apply(lambda x: max(1, x)).apply(np.log)
df_grouped

,lat_municipio_residencia,lon_municipio_residencia,cod_municipio_residencia,universidad,curso_academico,count,scaled_count,log_count
0,27.75379,-18.003640,38013,uam,2017-18,1,1.000000,0.000000
1,27.75379,-18.003640,38013,ucm,2017-18,5,1.003009,1.609438
2,27.75379,-18.003640,38013,ucm,2018-19,4,1.002257,1.386294
3,27.75379,-18.003640,38013,ucm,2019-20,4,1.002257,1.386294
4,27.75379,-18.003640,38013,ucm,2020-21,4,1.002257,1.386294
...,...,...,...,...,...,...,...,...
46021,43.73261,-7.674875,27064,uam,2022-23,1,1.000000,0.000000
46022,43.73261,-7.674875,27064,uc3m,2020-21,1,1.000000,0.000000
46023,43.73261,-7.674875,27064,uc3m,2021-22,1,1.000000,0.000000
46024,43.73261,-7.674875,27064,uc3m,2022-23,1,1.000000,0.000000


In [40]:
ine = pd.read_csv("INe.csv", sep=";", low_memory=False)
#Nos quedamos con la población total del 2017 en adelante
ine = ine[ine["Sexo"] == "Total"]
ine = ine[ine["Periodo"] >= 2017]
ine[['cod_municipio_residencia', 'municipio']] = ine['Municipios'].str.split(' ', n=1, expand=True)
ine['cod_municipio_residencia'] = ine['cod_municipio_residencia'].astype(int)
ine['curso_academico'] = ine['Periodo'].astype(int).apply(lambda x: f"{x}-{(x + 1) % 100:02d}")
ine = ine.dropna()
ine['Total'] = ine['Total'].str.replace('.', '', regex=False).astype(int)
ine['Total'] = ine['Total'].astype('int')
ine['poblacion'] = ine['Total'].apply(
    lambda x: 'Muy densa' if x > 500000 else
              'Densa' if x > 100000 else
              'Moderadamente densa' if x > 25000 else
              'Poco densa' if x > 5000 else
              'Muy poco densa'
)
ine = ine.drop(columns=['Municipios',"Sexo","Periodo", "Total"])
ine

,cod_municipio_residencia,municipio,curso_academico,poblacion
0,44001,Ababuj,2024-25,Muy poco densa
1,44001,Ababuj,2023-24,Muy poco densa
2,44001,Ababuj,2022-23,Muy poco densa
3,44001,Ababuj,2021-22,Muy poco densa
4,44001,Ababuj,2020-21,Muy poco densa
...,...,...,...,...
707922,4103,Zurgena,2021-22,Muy poco densa
707923,4103,Zurgena,2020-21,Muy poco densa
707924,4103,Zurgena,2019-20,Muy poco densa
707925,4103,Zurgena,2018-19,Muy poco densa


In [45]:
#Juntamos los conjuntos
df_merged = df_grouped.merge(ine, on=['cod_municipio_residencia', 'curso_academico'], how='left')
df_merged

,lat_municipio_residencia,lon_municipio_residencia,cod_municipio_residencia,universidad,curso_academico,count,scaled_count,log_count,municipio,poblacion
0,27.75379,-18.003640,38013,uam,2017-18,1,1.000000,0.000000,Frontera,Muy poco densa
1,27.75379,-18.003640,38013,ucm,2017-18,5,1.003009,1.609438,Frontera,Muy poco densa
2,27.75379,-18.003640,38013,ucm,2018-19,4,1.002257,1.386294,Frontera,Muy poco densa
3,27.75379,-18.003640,38013,ucm,2019-20,4,1.002257,1.386294,Frontera,Muy poco densa
4,27.75379,-18.003640,38013,ucm,2020-21,4,1.002257,1.386294,Frontera,Muy poco densa
...,...,...,...,...,...,...,...,...,...,...
46021,43.73261,-7.674875,27064,uam,2022-23,1,1.000000,0.000000,"Vicedo, O",Muy poco densa
46022,43.73261,-7.674875,27064,uc3m,2020-21,1,1.000000,0.000000,"Vicedo, O",Muy poco densa
46023,43.73261,-7.674875,27064,uc3m,2021-22,1,1.000000,0.000000,"Vicedo, O",Muy poco densa
46024,43.73261,-7.674875,27064,uc3m,2022-23,1,1.000000,0.000000,"Vicedo, O",Muy poco densa


In [46]:
rows_with_nan = ine[ine.isna().any(axis=1)]

print("Filas con valores NaN:")
print(rows_with_nan)

Filas con valores NaN:
Empty DataFrame
Columns: [cod_municipio_residencia, municipio, curso_academico, poblacion]
Index: []


In [47]:
df_merged.to_csv('matricula.csv', index=False)  # index=False para no guardar el índice